In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm_laplace as fmm
import numpy as np

In [2]:
fmm.__doc__

"exafmm's pybind11 module for Laplace kernel"

#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

In [5]:
sources = fmm.init_sources(src_coords, src_charges)
targets = fmm.init_targets(trg_coords)

#### set FMM parameters

In [6]:
p = 10       # expansion order
ncrit = 100  # max number of bodies allow per leaf box
fmm.configure(p, ncrit)

#### build tree

In [7]:
nodes = fmm.build_tree(sources, targets)

#### build interaction lists

In [8]:
fmm.build_list()

#### precompute

In [9]:
fmm.precompute()

#### evaluate potentials

In [10]:
potentials = fmm.evaluate()

P2M                  : 1.1798500e-01
M2M                  : 8.7798000e-02
P2L                  : 2.5130000e-03
M2P                  : 1.7340000e-03
P2P                  : 4.5462000e-02
M2L                  : 6.5573000e-01
L2L                  : 8.3873000e-02
L2P                  : 1.3397600e-01


In [11]:
potentials[::10000]

array([6548.90377095, 8533.81412822, 9131.76140651, 8068.85996499,
       7398.88588199, 8137.02006689, 7191.33871288, 6821.53451296,
       7364.76982343, 7256.16475538])

#### check accuracy

In [12]:
fmm.check_accuracy()

-------------- Error ---------------
Potential Error      : 3.3682917e-09
Gradient Error       : 2.3777205e-07


#### update charges and run FMM iteratively

In [13]:
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy

---------- iteration 0 ----------
P2M                  : 1.2372600e-01
M2M                  : 8.8194000e-02
P2L                  : 5.9000000e-05
M2P                  : 1.3580000e-03
P2P                  : 4.3553000e-02
M2L                  : 6.5925700e-01
L2L                  : 8.5429000e-02
L2P                  : 1.3093600e-01
-------------- Error ---------------
Potential Error      : 3.3299854e-09
Gradient Error       : 2.3396321e-07
---------- iteration 1 ----------
P2M                  : 1.1902700e-01
M2M                  : 9.2122000e-02
P2L                  : 1.9840000e-03
M2P                  : 2.9070000e-03
P2P                  : 4.4828000e-02
M2L                  : 6.5886000e-01
L2L                  : 8.4513000e-02
L2P                  : 1.3146500e-01
-------------- Error ---------------
Potential Error      : 3.2325451e-09
Gradient Error       : 2.2718251e-07
---------- iteration 2 ----------
P2M                  : 1.2336500e-01
M2M                  : 8.9268000e-02
P2L       